In [14]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import re
import nltk
import nltk

In [13]:
data = pd.read_csv("Twitter_Data.csv")
print(data.head())

                                          clean_text  category
0  when modi promised “minimum government maximum...      -1.0
1  talk all the nonsense and continue all the dra...       0.0
2  what did just say vote for modi  welcome bjp t...       1.0
3  asking his supporters prefix chowkidar their n...       1.0
4  answer who among these the most powerful world...       1.0


In [28]:
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
stopword=set(stopwords.words('english'))
def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
data["cleaned_text"] = data["clean_text"].apply(clean)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sruja\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
data['sentence_length'] = data['cleaned_text'].apply(lambda x: len(x.split()))

In [5]:
X = data['cleaned_text']
y = data['category']

In [6]:
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)

X_encoded = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_encoded, padding='pre')

vocab_size = len(tokenizer.word_index) + 1
max_length = X_padded.shape[1]

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(LSTM(100))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
y_encoded = pd.get_dummies(y)

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

In [18]:
model.fit(X_train, y_train, epochs=10, batch_size=64)

Epoch 1/10
2038/2038 [==============================] - 333s 163ms/step - loss: 0.1393 - accuracy: 0.9514
Epoch 2/10
2038/2038 [==============================] - 281s 138ms/step - loss: 0.1066 - accuracy: 0.9622
Epoch 3/10
2038/2038 [==============================] - 414s 203ms/step - loss: 0.0862 - accuracy: 0.9697
Epoch 4/10
2038/2038 [==============================] - 486s 238ms/step - loss: 0.0686 - accuracy: 0.9756
Epoch 5/10
2038/2038 [==============================] - 295s 145ms/step - loss: 0.0559 - accuracy: 0.9804
Epoch 6/10
2038/2038 [==============================] - 314s 154ms/step - loss: 0.0476 - accuracy: 0.9832
Epoch 7/10
2038/2038 [==============================] - 253s 124ms/step - loss: 0.0375 - accuracy: 0.9868
Epoch 8/10
2038/2038 [==============================] - 263s 129ms/step - loss: 0.0331 - accuracy: 0.9887
Epoch 9/10
2038/2038 [==============================] - 277s 136ms/step - loss: 0.0287 - accuracy: 0.9899
Epoch 10/10
2038/2038 [=======================

In [22]:
y_pred = model.predict(X_test)

1019/1019 [==============================] - 8s 7ms/step


In [25]:
y_pred_normalized = pd.DataFrame(y_pred).applymap(lambda x: 1 if x >= 0.5 else 0)

In [26]:
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(y_test.values.argmax(axis=1), y_pred_normalized.values.argmax(axis=1)))
accuracy = accuracy_score(y_test.values.argmax(axis=1), y_pred_normalized.values.argmax(axis=1))
print("Accuracy:", accuracy)

              precision    recall  f1-score   support

           0       0.74      0.73      0.73      7179
           1       0.84      0.83      0.83     11034
           2       0.84      0.85      0.84     14383

    accuracy                           0.81     32596
   macro avg       0.80      0.80      0.80     32596
weighted avg       0.81      0.81      0.81     32596

Accuracy: 0.8142716897778869
